In [8]:
import pandas as pd
import os
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mode
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [9]:
def _log(str):
    os.system(f'echo \"{str}\"')
    print(str)

In [90]:
NAN = '__NAN__'
INPUT_ROOT = '../input/data-science-bowl-2019'
JOIN_KEY = ['installation_id', 'game_session', 'title']
TARGET = 'accuracy_group'
FEATURES = {
    'event_id', 
    'game_session', 
    'timestamp', 
    'installation_id', 
    'event_count',
    'event_code', 
    'game_time', 
    'title', 
    'type', 
    'world'
}
CATEGORICAL_FEATURES = {
    'event_id', 
    'game_session',
    'timestamp',
    'installation_id',
    'event_code',
    'title', 
    'type', 
    'world'
}
EVENT_CODES = ['2000', '2010', '2020', '2025', '2030', '2035', '2040', '2050', '2060', '2070', '2075', '2080', '2081', '2083', '3010', '3020', '3021', '3110', '3120', '3121', '4010', '4020', '4021', '4022', '4025', '4030', '4031', '4035', '4040', '4045', '4050', '4070', '4080', '4090', '4095', '4100', '4110', '4220', '4230', '4235', '5000', '5010']

In [89]:
def _init():
    # Characters such as empty strings '' or numpy.inf are considered NA values
    pd.set_option('use_inf_as_na', True)
    pd.set_option('display.max_columns', 999)
    pd.set_option('display.max_rows', 999)
    
    
_init()

In [12]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk(INPUT_ROOT):
    for filename in filenames:
        _log(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/data-science-bowl-2019\test.csv
../input/data-science-bowl-2019\test.csv.zip
../input/data-science-bowl-2019\train.csv
../input/data-science-bowl-2019\train.csv.zip
../input/data-science-bowl-2019\train_labels.csv
../input/data-science-bowl-2019\train_labels.csv.zip


In [13]:
%%time
train_raw = pd.read_csv(f'{INPUT_ROOT}/train.csv', usecols=FEATURES)
train_labels = pd.read_csv(f'{INPUT_ROOT}/train_labels.csv', usecols=JOIN_KEY + [TARGET])
test_raw = pd.read_csv(f'{INPUT_ROOT}/test.csv', usecols=FEATURES)
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11341042 entries, 0 to 11341041
Data columns (total 10 columns):
event_id           object
game_session       object
timestamp          object
installation_id    object
event_count        int64
event_code         int64
game_time          int64
title              object
type               object
world              object
dtypes: int64(3), object(7)
memory usage: 865.3+ MB
Wall time: 33.8 s


In [14]:
train_labels.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17690 entries, 0 to 17689
Data columns (total 4 columns):
game_session       17690 non-null object
installation_id    17690 non-null object
title              17690 non-null object
accuracy_group     17690 non-null int64
dtypes: int64(1), object(3)
memory usage: 552.9+ KB


In [15]:
test_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156414 entries, 0 to 1156413
Data columns (total 10 columns):
event_id           1156414 non-null object
game_session       1156414 non-null object
timestamp          1156414 non-null object
installation_id    1156414 non-null object
event_count        1156414 non-null int64
event_code         1156414 non-null int64
game_time          1156414 non-null int64
title              1156414 non-null object
type               1156414 non-null object
world              1156414 non-null object
dtypes: int64(3), object(7)
memory usage: 88.2+ MB


In [16]:
vs = sorted(train_raw['type'].unique())
_log(f'{len(vs)} train_raw type={vs}')

4 train_raw type=['Activity', 'Assessment', 'Clip', 'Game']


In [17]:
vs = sorted(train_raw['world'].unique())
_log(f'{len(vs)} train_raw type={vs}')

4 train_raw type=['CRYSTALCAVES', 'MAGMAPEAK', 'NONE', 'TREETOPCITY']


In [18]:
vs = sorted(train_raw['event_code'].unique())
_log(f'{len(vs)} train_raw type={vs}')

42 train_raw type=[2000, 2010, 2020, 2025, 2030, 2035, 2040, 2050, 2060, 2070, 2075, 2080, 2081, 2083, 3010, 3020, 3021, 3110, 3120, 3121, 4010, 4020, 4021, 4022, 4025, 4030, 4031, 4035, 4040, 4045, 4050, 4070, 4080, 4090, 4095, 4100, 4110, 4220, 4230, 4235, 5000, 5010]


In [19]:
vs = sorted(train_raw['title'].unique())
_log(f'{len(vs)} train_raw titles={vs}')

44 train_raw titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [20]:
vs = sorted(test_raw['title'].unique())
_log(f'{len(vs)} test titles={vs}')

44 test titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [21]:
vs = sorted(train_labels['title'].unique())
_log(f'{len(vs)} train_labels titles={vs}')

5 train_labels titles=['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']


# All event ids in test set also exist in train set

In [44]:
test_set = set(test_raw['event_id'])
train_set = set(train_raw['event_id'])
vs = test_set - train_set
_log(f'{len(vs)} event_ids exist in test set but not train set.')

0 event_ids exist in test set but not train set.


In [22]:
def _remove_unlabelled_data(train_raw, train_labels):
    return train_raw[train_raw['installation_id'].isin(train_labels['installation_id'].unique())]


train_raw = _remove_unlabelled_data(train_raw, train_labels)

In [23]:
%%time
def _add_labels(train_raw, train_labels, on):
    return pd.merge(train_raw, train_labels, on=on, how='left')


train_raw = _add_labels(train_raw, train_labels, on=JOIN_KEY)

#_log(f'train[\'{TARGET}\'] count null={train[TARGET].isna().sum()}')

Wall time: 4.02 s


In [24]:
def _transform_timestamp(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df


train_raw = _transform_timestamp(train_raw)
test_raw = _transform_timestamp(test_raw)

In [25]:
def _drop_unused_columns(df):
    cols = ['game_session', 'type']
    return df.drop(columns=cols)

    
#train = _drop_unused_columns(train)
#test = _drop_unused_columns(test)

In [26]:
%%time
def _set_string_type(df, cols):
    df[cols] = df[cols].fillna(NAN).astype(str)
    return df


cols = list(CATEGORICAL_FEATURES)
train_raw = _set_string_type(train_raw, cols=cols + [TARGET])
test_raw = _set_string_type(test_raw, cols=cols)

Wall time: 1min 35s


In [27]:
def _sort_it(df):
    return df.sort_values(by=['installation_id', 'timestamp'])


train_raw = _sort_it(train_raw)
test_raw = _sort_it(test_raw)

# Multiple accuracy groups per installation id
In the train set, there are multiple accuracy groups per installation id. The task is to predict the accuracy group of the **last** assessment for a given installation id.

In [28]:
vs = train_raw[train_raw[TARGET] != NAN].groupby('installation_id')[TARGET].nunique()
vs

installation_id
0006a69f    3
0006c192    3
00129856    1
001d0ed0    3
00225f67    1
           ..
ff9305d7    2
ff9715db    4
ffc90c32    2
ffd2871d    1
ffeb0b1b    2
Name: accuracy_group, Length: 3614, dtype: int64

In [29]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7734558 entries, 0 to 7734557
Data columns (total 11 columns):
event_id           object
game_session       object
timestamp          object
installation_id    object
event_count        int64
event_code         object
game_time          int64
title              object
type               object
world              object
accuracy_group     object
dtypes: int64(2), object(9)
memory usage: 708.1+ MB


In [30]:
def _log_smoothing(df, cols):
    for col in cols:
        df[col] = np.log(df[col] + 1)
    return df


#cols = ['event_count', 'game_time']
#train = _log_smoothing(train, cols)
#test = _log_smoothing(test, cols)

In [31]:
train_raw.head(40)

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world,accuracy_group
0,27253bdc,34ba1a28d02ba8ba,2019-08-06 04:57:18.904000+00:00,0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,__NAN__
1,27253bdc,4b57c9a59474a1b9,2019-08-06 04:57:45.301000+00:00,0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,__NAN__
2,77261ab5,2b9d5af79bcdb79f,2019-08-06 04:58:14.538000+00:00,0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__
3,b2dba42b,2b9d5af79bcdb79f,2019-08-06 04:58:14.615000+00:00,0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__
4,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:16.680000+00:00,0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__
5,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:18.474000+00:00,0006a69f,4,4070,3937,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__
6,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:19.365000+00:00,0006a69f,5,4070,4820,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__
7,1bb5fbdb,2b9d5af79bcdb79f,2019-08-06 04:58:21.490000+00:00,0006a69f,6,3110,6954,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__
8,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:22.732000+00:00,0006a69f,7,4070,8187,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__
9,5e812b27,2b9d5af79bcdb79f,2019-08-06 04:58:23.295000+00:00,0006a69f,8,4030,8745,Sandcastle Builder (Activity),Activity,MAGMAPEAK,__NAN__


In [32]:
train_raw.describe(include='all')

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world,accuracy_group
count,7734558,7734558,7734558,7734558,7.734558e+06,7734558,7.734558e+06,7734558,7734558,7734558,7734558
unique,379,175467,7347901,3614,NaN,42,NaN,44,4,4,5
top,bb3e370b,bb1f09ec062b6660,2019-09-03 17:03:39.779000+00:00,f1c21eda,NaN,4070,NaN,Bottle Filler (Activity),Game,MAGMAPEAK,__NAN__
freq,168955,3182,10,58988,NaN,1521039,NaN,661294,3834750,3208242,6869111
mean,NaN,NaN,NaN,NaN,9.775152e+01,NaN,1.677046e+05,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,1.311293e+02,NaN,1.068495e+06,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.400000e+01,NaN,2.927100e+04,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,5.700000e+01,NaN,6.941200e+04,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.210000e+02,NaN,1.479110e+05,NaN,NaN,NaN,NaN


In [33]:
test_raw.head()

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10 16:50:24.910000+00:00,00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10 16:50:55.503000+00:00,00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10 16:51:51.805000+00:00,00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10 16:53:12.825000+00:00,00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10 16:54:12.115000+00:00,00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [34]:
test_raw.describe(include='all')

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world
count,1156414,1156414,1156414,1156414,1.156414e+06,1156414,1.156414e+06,1156414,1156414,1156414
unique,365,28445,1103166,1000,NaN,42,NaN,44,4,4
top,bb3e370b,b246933e8ac6a4aa,2019-09-13 18:38:38.848000+00:00,7b728c89,NaN,4070,NaN,Bottle Filler (Activity),Game,MAGMAPEAK
freq,28667,4456,9,21237,NaN,234260,NaN,112223,572260,511291
mean,NaN,NaN,NaN,NaN,1.107075e+02,NaN,1.869646e+05,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,2.193408e+02,NaN,1.670162e+06,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.500000e+01,NaN,3.056125e+04,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,6.000000e+01,NaN,7.102800e+04,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.240000e+02,NaN,1.476910e+05,NaN,NaN,NaN


# Train-test split not by time
Both train and test sets span the same time period.

In [35]:
_log(f'train[timestamp] is from {train_raw.timestamp.min()} to {train_raw.timestamp.max()}')
_log(f'test[timestamp] is from {test_raw.timestamp.min()} to {test_raw.timestamp.max()}')

train[timestamp] is from 2019-07-23 14:38:25.256000+00:00 to 2019-10-14 21:57:26.930000+00:00
test[timestamp] is from 2019-07-24 00:04:25.361000+00:00 to 2019-10-14 21:00:34.858000+00:00


# Feature Engineering

In [92]:
def _cutoff(df, TARGET):
    return df[df[TARGET] != NAN]['timestamp'].max()


def _trim_events_after_last_assessment(df, cutoff):
    res = df[df['timestamp'] <= cutoff]
    #_log(f'cutoff: before={df.shape}, after={res.shape}')
    return res
    
    
def _target_variable(df, cutoff, TARGET):
    vs = df[df['timestamp'] == cutoff][TARGET].values
    assert len(set(vs)) == 1
    return str(int(float(vs[0])))
    
    
def _game_time_median(df):
    return df.groupby(['game_session'])['game_time'].transform('max').median()
    
    
def _event_count_median(df):
    return df.groupby(['game_session'])['event_count'].transform('max').median()


def _game_time_last_session(df):
    return df[df['timestamp'] == df['timestamp'].max()]['game_time'].max()


def _event_count_last_session(df):
    return df[df['timestamp'] == df['timestamp'].max()]['event_count'].max()


def _event_code_counts(df, code):
    total = np.int32([sum(df['event_code'] == code)])
    activity = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Activity'))])
    assessment = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Assessment'))])
    clip = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Clip'))])
    game = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Game'))])
    return (total, activity, assessment, clip, game)


def _features(df, installation_id, EVENT_CODES):
    res = {}
    iid = df[df['installation_id'] == installation_id]
    if TARGET in df.columns:
        cutoff = _cutoff(iid, TARGET)
        iid = _trim_events_after_last_assessment(iid, cutoff)
        res[TARGET] = _target_variable(iid, cutoff, TARGET)
    res['installation_id'] = [installation_id]
    res['type_activity'] = np.int32([sum(iid['type'] == 'Activity')])
    res['type_assessment'] = np.int32([sum(iid['type'] == 'Assessment')])
    res['type_clip'] = np.int32([sum(iid['type'] == 'Clip')])
    res['type_game'] = np.int32([sum(iid['type'] == 'Game')])
    assert len(iid) == res['type_activity'][0] + res['type_assessment'][0] + res['type_clip'][0] + res['type_game'][0]
    res['world_crystalcaves'] = np.int32([sum(iid['world'] == 'CRYSTALCAVES')])
    res['world_magmapeak'] = np.int32([sum(iid['world'] == 'MAGMAPEAK')])
    res['world_treetopcity'] = np.int32([sum(iid['world'] == 'TREETOPCITY')])
    res['world_none'] = np.int32([sum(iid['world'] == 'NONE')])
    res['title_12_monkeys'] = np.int32([sum(iid['title'] == '12 Monkeys')])
    res['title_air_show'] = np.int32([sum(iid['title'] == 'Air Show')])
    res['title_all_star_sorting'] = np.int32([sum(iid['title'] == 'All Star Sorting')])
    res['title_balancing_act'] = np.int32([sum(iid['title'] == 'Balancing Act')])
    res['title_bird_measurer'] = np.int32([sum(iid['title'] == 'Bird Measurer (Assessment)')])
    res['title_bottle_filler'] = np.int32([sum(iid['title'] == 'Bottle Filler (Activity)')])
    res['title_bubble_bath'] = np.int32([sum(iid['title'] == 'Bubble Bath')])
    res['title_bug_measurer'] = np.int32([sum(iid['title'] == 'Bug Measurer (Activity)')])
    res['title_cart_balancer'] = np.int32([sum(iid['title'] == 'Cart Balancer (Assessment)')])
    res['title_cauldron_filler'] = np.int32([sum(iid['title'] == 'Cauldron Filler (Assessment)')])
    res['title_chest_sorter'] = np.int32([sum(iid['title'] == 'Chest Sorter (Assessment)')])
    res['title_chicken_balancer'] = np.int32([sum(iid['title'] == 'Chicken Balancer (Activity)')])
    res['title_chow_time'] = np.int32([sum(iid['title'] == 'Chow Time')])
    res['title_costume_box'] = np.int32([sum(iid['title'] == 'Costume Box')])
    res['title_crystal_caves_1'] = np.int32([sum(iid['title'] == 'Crystal Caves - Level 1')])
    res['title_crystal_caves_2'] = np.int32([sum(iid['title'] == 'Crystal Caves - Level 2')])
    res['title_crystal_caves_3'] = np.int32([sum(iid['title'] == 'Crystal Caves - Level 3')])
    res['title_crystals_rule'] = np.int32([sum(iid['title'] == 'Crystals Rule')])
    res['title_dino_dive'] = np.int32([sum(iid['title'] == 'Dino Dive')])
    res['title_dino_drink'] = np.int32([sum(iid['title'] == 'Dino Drink')])
    res['title_egg_dropper'] = np.int32([sum(iid['title'] == 'Egg Dropper (Activity)')])
    res['title_fireworks'] = np.int32([sum(iid['title'] == 'Fireworks (Activity)')])
    res['title_flower_waterer'] = np.int32([sum(iid['title'] == 'Flower Waterer (Activity)')])
    res['title_happy_camel'] = np.int32([sum(iid['title'] == 'Happy Camel')])
    res['title_heavy'] = np.int32([sum(iid['title'] == 'Heavy, Heavier, Heaviest')])
    res['title_honey_cake'] = np.int32([sum(iid['title'] == 'Honey Cake')])
    res['title_leaf_leader'] = np.int32([sum(iid['title'] == 'Leaf Leader')])
    res['title_lifting'] = np.int32([sum(iid['title'] == 'Lifting Heavy Things')])
    res['title_magma_peak_1'] = np.int32([sum(iid['title'] == 'Magma Peak - Level 1')])
    res['title_magma_peak_2'] = np.int32([sum(iid['title'] == 'Magma Peak - Level 2')])
    res['title_mushroom_sorter'] = np.int32([sum(iid['title'] == 'Mushroom Sorter (Assessment)')])
    res['title_ordering_spheres'] = np.int32([sum(iid['title'] == 'Ordering Spheres')])
    res['title_pan_balance'] = np.int32([sum(iid['title'] == 'Pan Balance')])
    res['title_pirate_tale'] = np.int32([sum(iid['title'] == "Pirate's Tale")])
    res['title_rulers'] = np.int32([sum(iid['title'] == 'Rulers')])
    res['title_sandcastle'] = np.int32([sum(iid['title'] == 'Sandcastle Builder (Activity)')])
    res['title_scrub'] = np.int32([sum(iid['title'] == 'Scrub-A-Dub')])
    res['title_slop'] = np.int32([sum(iid['title'] == 'Slop Problem')])
    res['title_treasure_map'] = np.int32([sum(iid['title'] == 'Treasure Map')])
    res['title_treetop_city_1'] = np.int32([sum(iid['title'] == 'Tree Top City - Level 1')])
    res['title_treetop_city_2'] = np.int32([sum(iid['title'] == 'Tree Top City - Level 2')])
    res['title_treetop_city_3'] = np.int32([sum(iid['title'] == 'Tree Top City - Level 3')])
    res['title_watering_hole'] = np.int32([sum(iid['title'] == 'Watering Hole (Activity)')])
    res['title_welcome'] = np.int32([sum(iid['title'] == 'Welcome to Lost Lagoon!')])
    
    for code in EVENT_CODES:
        (total, activity, assessment, clip, game) = _event_code_counts(iid, code)
        res[f'event_{code}'] = total
        res[f'event_{code}_activity'] = activity
        res[f'event_{code}_assessment'] = assessment
        res[f'event_{code}_clip'] = clip
        res[f'event_{code}_game'] = game
    
    res['game_time_p50'] = np.int32(_game_time_median(iid))
    res['event_count_p50'] = np.int32(_event_count_median(iid))
    res['game_time_last_session'] = np.int32(_game_time_last_session(iid))
    res['event_count_last_session'] = np.int32(_event_count_last_session(iid))
    return pd.DataFrame.from_dict(res)


def _preprocess(raw, EVENT_CODES):
    res = pd.DataFrame()
    iids = raw['installation_id'].unique()[:10]
    for iid in tqdm(iids):
        res = pd.concat([res, _features(raw, iid, EVENT_CODES)])
    return res


train = _preprocess(train_raw, EVENT_CODES)
train.info(max_cols=999)

100%|██████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.43it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 0
Data columns (total 268 columns):
accuracy_group              10 non-null object
installation_id             10 non-null object
type_activity               10 non-null int32
type_assessment             10 non-null int32
type_clip                   10 non-null int32
type_game                   10 non-null int32
world_crystalcaves          10 non-null int32
world_magmapeak             10 non-null int32
world_treetopcity           10 non-null int32
world_none                  10 non-null int32
title_12_monkeys            10 non-null int32
title_air_show              10 non-null int32
title_all_star_sorting      10 non-null int32
title_balancing_act         10 non-null int32
title_bird_measurer         10 non-null int32
title_bottle_filler         10 non-null int32
title_bubble_bath           10 non-null int32
title_bug_measurer          10 non-null int32
title_cart_balancer         10 non-null int32
title_cauldron_filler

In [93]:
train.head(10)

,accuracy_group,installation_id,type_activity,type_assessment,type_clip,type_game,world_crystalcaves,world_magmapeak,world_treetopcity,world_none,title_12_monkeys,title_air_show,title_all_star_sorting,title_balancing_act,title_bird_measurer,title_bottle_filler,title_bubble_bath,title_bug_measurer,title_cart_balancer,title_cauldron_filler,title_chest_sorter,title_chicken_balancer,title_chow_time,title_costume_box,title_crystal_caves_1,title_crystal_caves_2,title_crystal_caves_3,title_crystals_rule,title_dino_dive,title_dino_drink,title_egg_dropper,title_fireworks,title_flower_waterer,title_happy_camel,title_heavy,title_honey_cake,title_leaf_leader,title_lifting,title_magma_peak_1,title_magma_peak_2,title_mushroom_sorter,title_ordering_spheres,title_pan_balance,title_pirate_tale,title_rulers,title_sandcastle,title_scrub,title_slop,title_treasure_map,title_treetop_city_1,title_treetop_city_2,title_treetop_city_3,title_watering_hole,title_welcome,event_2000,event_2000_activity,event_2000_assessment,event_2000_clip,event_2000_game,event_2010,event_2010_activity,event_2010_assessment,event_2010_clip,event_2010_game,event_2020,event_2020_activity,event_2020_assessment,event_2020_clip,event_2020_game,event_2025,event_2025_activity,event_2025_assessment,event_2025_clip,event_2025_game,event_2030,event_2030_activity,event_2030_assessment,event_2030_clip,event_2030_game,event_2035,event_2035_activity,event_2035_assessment,event_2035_clip,event_2035_game,event_2040,event_2040_activity,event_2040_assessment,event_2040_clip,event_2040_game,event_2050,event_2050_activity,event_2050_assessment,event_2050_clip,event_2050_game,event_2060,event_2060_activity,event_2060_assessment,event_2060_clip,event_2060_game,event_2070,event_2070_activity,event_2070_assessment,event_2070_clip,event_2070_game,event_2075,event_2075_activity,event_2075_assessment,event_2075_clip,event_2075_game,event_2080,event_2080_activity,event_2080_assessment,event_2080_clip,event_2080_game,event_2081,event_2081_activity,event_2081_assessment,event_2081_clip,event_2081_game,event_2083,event_2083_activity,event_2083_assessment,event_2083_clip,event_2083_game,event_3010,event_3010_activity,event_3010_assessment,event_3010_clip,event_3010_game,event_3020,event_3020_activity,event_3020_assessment,event_3020_clip,event_3020_game,event_3021,event_3021_activity,event_3021_assessment,event_3021_clip,event_3021_game,event_3110,event_3110_activity,event_3110_assessment,event_3110_clip,event_3110_game,event_3120,event_3120_activity,event_3120_assessment,event_3120_clip,event_3120_game,event_3121,event_3121_activity,event_3121_assessment,event_3121_clip,event_3121_game,event_4010,event_4010_activity,event_4010_assessment,event_4010_clip,event_4010_game,event_4020,event_4020_activity,event_4020_assessment,event_4020_clip,event_4020_game,event_4021,event_4021_activity,event_4021_assessment,event_4021_clip,event_4021_game,event_4022,event_4022_activity,event_4022_assessment,event_4022_clip,event_4022_game,event_4025,event_4025_activity,event_4025_assessment,event_4025_clip,event_4025_game,event_4030,event_4030_activity,event_4030_assessment,event_4030_clip,event_4030_game,event_4031,event_4031_activity,event_4031_assessment,event_4031_clip,event_4031_game,event_4035,event_4035_activity,event_4035_assessment,event_4035_clip,event_4035_game,event_4040,event_4040_activity,event_4040_assessment,event_4040_clip,event_4040_game,event_4045,event_4045_activity,event_4045_assessment,event_4045_clip,event_4045_game,event_4050,event_4050_activity,event_4050_assessment,event_4050_clip,event_4050_game,event_4070,event_4070_activity,event_4070_assessment,event_4070_clip,event_4070_game,event_4080,event_4080_activity,event_4080_assessment,event_4080_clip,event_4080_game,event_4090,event_4090_activity,event_4090_assessment,event_4090_clip,event_4090_game,event_4095,event_4095_activity,event_4095_assessment,event_4095_clip,event_4095_game,event_4100,event_4100_activity,event_4100_assessment,event_4100_clip

In [94]:
test = _preprocess(test_raw, EVENT_CODES)
test.info(max_cols=999)

100%|██████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.43it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 0
Data columns (total 267 columns):
installation_id             10 non-null object
type_activity               10 non-null int32
type_assessment             10 non-null int32
type_clip                   10 non-null int32
type_game                   10 non-null int32
world_crystalcaves          10 non-null int32
world_magmapeak             10 non-null int32
world_treetopcity           10 non-null int32
world_none                  10 non-null int32
title_12_monkeys            10 non-null int32
title_air_show              10 non-null int32
title_all_star_sorting      10 non-null int32
title_balancing_act         10 non-null int32
title_bird_measurer         10 non-null int32
title_bottle_filler         10 non-null int32
title_bubble_bath           10 non-null int32
title_bug_measurer          10 non-null int32
title_cart_balancer         10 non-null int32
title_cauldron_filler       10 non-null int32
title_chest_sorter    

In [95]:
test.head(10)

,installation_id,type_activity,type_assessment,type_clip,type_game,world_crystalcaves,world_magmapeak,world_treetopcity,world_none,title_12_monkeys,title_air_show,title_all_star_sorting,title_balancing_act,title_bird_measurer,title_bottle_filler,title_bubble_bath,title_bug_measurer,title_cart_balancer,title_cauldron_filler,title_chest_sorter,title_chicken_balancer,title_chow_time,title_costume_box,title_crystal_caves_1,title_crystal_caves_2,title_crystal_caves_3,title_crystals_rule,title_dino_dive,title_dino_drink,title_egg_dropper,title_fireworks,title_flower_waterer,title_happy_camel,title_heavy,title_honey_cake,title_leaf_leader,title_lifting,title_magma_peak_1,title_magma_peak_2,title_mushroom_sorter,title_ordering_spheres,title_pan_balance,title_pirate_tale,title_rulers,title_sandcastle,title_scrub,title_slop,title_treasure_map,title_treetop_city_1,title_treetop_city_2,title_treetop_city_3,title_watering_hole,title_welcome,event_2000,event_2000_activity,event_2000_assessment,event_2000_clip,event_2000_game,event_2010,event_2010_activity,event_2010_assessment,event_2010_clip,event_2010_game,event_2020,event_2020_activity,event_2020_assessment,event_2020_clip,event_2020_game,event_2025,event_2025_activity,event_2025_assessment,event_2025_clip,event_2025_game,event_2030,event_2030_activity,event_2030_assessment,event_2030_clip,event_2030_game,event_2035,event_2035_activity,event_2035_assessment,event_2035_clip,event_2035_game,event_2040,event_2040_activity,event_2040_assessment,event_2040_clip,event_2040_game,event_2050,event_2050_activity,event_2050_assessment,event_2050_clip,event_2050_game,event_2060,event_2060_activity,event_2060_assessment,event_2060_clip,event_2060_game,event_2070,event_2070_activity,event_2070_assessment,event_2070_clip,event_2070_game,event_2075,event_2075_activity,event_2075_assessment,event_2075_clip,event_2075_game,event_2080,event_2080_activity,event_2080_assessment,event_2080_clip,event_2080_game,event_2081,event_2081_activity,event_2081_assessment,event_2081_clip,event_2081_game,event_2083,event_2083_activity,event_2083_assessment,event_2083_clip,event_2083_game,event_3010,event_3010_activity,event_3010_assessment,event_3010_clip,event_3010_game,event_3020,event_3020_activity,event_3020_assessment,event_3020_clip,event_3020_game,event_3021,event_3021_activity,event_3021_assessment,event_3021_clip,event_3021_game,event_3110,event_3110_activity,event_3110_assessment,event_3110_clip,event_3110_game,event_3120,event_3120_activity,event_3120_assessment,event_3120_clip,event_3120_game,event_3121,event_3121_activity,event_3121_assessment,event_3121_clip,event_3121_game,event_4010,event_4010_activity,event_4010_assessment,event_4010_clip,event_4010_game,event_4020,event_4020_activity,event_4020_assessment,event_4020_clip,event_4020_game,event_4021,event_4021_activity,event_4021_assessment,event_4021_clip,event_4021_game,event_4022,event_4022_activity,event_4022_assessment,event_4022_clip,event_4022_game,event_4025,event_4025_activity,event_4025_assessment,event_4025_clip,event_4025_game,event_4030,event_4030_activity,event_4030_assessment,event_4030_clip,event_4030_game,event_4031,event_4031_activity,event_4031_assessment,event_4031_clip,event_4031_game,event_4035,event_4035_activity,event_4035_assessment,event_4035_clip,event_4035_game,event_4040,event_4040_activity,event_4040_assessment,event_4040_clip,event_4040_game,event_4045,event_4045_activity,event_4045_assessment,event_4045_clip,event_4045_game,event_4050,event_4050_activity,event_4050_assessment,event_4050_clip,event_4050_game,event_4070,event_4070_activity,event_4070_assessment,event_4070_clip,event_4070_game,event_4080,event_4080_activity,event_4080_assessment,event_4080_clip,event_4080_game,event_4090,event_4090_activity,event_4090_assessment,event_4090_clip,event_4090_game,event_4095,event_4095_activity,event_4095_assessment,event_4095_clip,event_4095_game,event_4100,event_4100_activity,event_4100_assessment,event_4100_clip,event_4100_gam

In [96]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
_log(os.listdir("."))

['.ipynb_checkpoints', 'dsbowl19-preprocess.ipynb', 'test.csv', 'train.csv']
